In [1]:
import json
import os

import sys
from transformers import DonutProcessor

PUBTABLES_ANN = '../../pubtables-1m/Donut_Annotations_PT/minimal/Val/'
PUBTABLES_IMG = '../../pubtables-1m/PubTables-1M-Structure/Val/'

PUBTABLES_ANN = '../../pubtables-1m/Donut_Annotations_PT/minimal/Val/'
PUBTABLES_IMG = '../../pubtables-1m/PubTables-1M-Structure/Val/'

PUBTAB_ANN = '../../pubtabnet/anns/train/'
PUBTAB_IMG = '../../pubtabnet/imgs/train/'

ANN_PATH = PUBTAB_ANN
IMAGE_PATH = PUBTAB_IMG
IMG_FORMAT = '.png'

with open(ANN_PATH[:-1] + '_trunc_filelist.json') as file:
    json_list = file.read().splitlines()
    
json_list = [item for item in json_list]

In [2]:
aux_list = []

for json_item in json_list:
    aux_list.append(json_item[:-5])

json_list = aux_list

In [3]:
json_list[:10]

['PMC4841970_011_00',
 'PMC3163532_003_00',
 'PMC5397534_007_00',
 'PMC2996406_002_00',
 'PMC3492782_004_00',
 'PMC5094138_005_00',
 'PMC4112620_005_00',
 'PMC3880972_009_01',
 'PMC3759865_002_00',
 'PMC3588668_002_01']

In [4]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from transformers import VisionEncoderDecoderConfig
import torch

config = VisionEncoderDecoderConfig.from_pretrained("naver-clova-ix/donut-base")

image_size = [750, 750]
max_length = 1400#config.decoder.max_position_embeddings


config.encoder.image_size = image_size
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")

2024-03-15 13:26:20.411667: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 13:26:20.411694: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 13:26:20.411711: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-15 13:26:21.083991: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [5]:
from transformers import AddedToken

new_tokens = ["<table_extraction>", "<table>", "<row>", "<cell>", "<row_and_col_header>", "<row_header>", "<col_header>"]
new_tokens += ["<content_row_and_col_header>", "<content_row_header>", "<content_col_header>", "<content>"]
new_tokens += ["1", "≥","≤"]
new_tokens += ["<b>", "</b>", "<i>", "</i>", "<sup>", "</sup>","<sub>", "</sub>"]

for i in range(2):
    for j in range(2):
        for k in range(2):
            new_tokens.append("<span_type=0" + str(i) + str(j) + str(k) + ">")
            new_tokens.append("<span_type=1" + str(i) + str(j) + str(k) + ">")

new_tokens = [AddedToken(tag, rstrip = True, lstrip=True, normalized=False) for tag in new_tokens]

processor.tokenizer.add_tokens(new_tokens, special_tokens = False)

38

In [6]:
cell_types = ["<cell>", "<row_and_col_header>", "<row_header>", "<col_header>"]
for i in range(2):
    for j in range(2):
        for k in range(2):
            cell_types.append("<span_type=0" + str(i) + str(j) + str(k) + ">")
            cell_types.append("<span_type=1" + str(i) + str(j) + str(k) + ">")


cell_tokens = [processor.tokenizer.convert_tokens_to_ids([cell_type])[0] for cell_type in cell_types]
row_tokens = [processor.tokenizer.convert_tokens_to_ids([row_type])[0] for row_type in ['<row>', '</s>']]

In [7]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base", config=config, ignore_mismatched_sizes=True)

In [8]:
from torch import nn
from typing import Optional, Tuple, List, Union
from collections import defaultdict 

class DimPositionalEmbedding(nn.Module):
    """
    This module learns n sets positional embeddings up to n fixed maximum sizes.
    """

    def __init__(self, offset, dim_counters, max_dim_lens, embeddings_d):
        super().__init__()
        
        self.offset = offset
        self.embeddings_d = embeddings_d
        self.counter_dim = len(dim_counters)+1
        
        embeddings_list = []
        for i in range(len(max_dim_lens)):
            embeddings_list.append(nn.Embedding(max_dim_lens[i], embeddings_d))
        
        self.embeddings = nn.ModuleList(embeddings_list)
        
        self.counter_keys = defaultdict(lambda: 0) 
        for i in range(self.counter_dim-1):
            for counter_key in dim_counters[i]:
                self.counter_keys[counter_key] = i+1

                
    def get_sequence_embeddings_map(self, input_ids:torch.Tensor, counters_state:List = None):
        seq_len = input_ids.shape[0]
        embeddings_map = [[0 for i in range(seq_len)] for j in range(self.counter_dim)]
        
        if counters_state == None:
            counters = [0 for i in range(self.counter_dim)]
            counters[0] = self.offset
        else:
            counters = counters_state
        
        for i, token in enumerate(input_ids):
            counter = self.counter_keys[int(token)]
            for j in range(counter):
                counters[j] = 0
            counters[counter] += 1
            
            for j in range(self.counter_dim):
                embeddings_map[j][i] = counters[j]
            
        return embeddings_map, counters
    
    
    def get_embeddings_sum(self, index_lists:List):
        embeddings_sum = torch.zeros(( len(index_lists[0], self.embeddings_d)), device = self.embeddings[0].weight.device)
        for i, index_list in enumerate(index_lists):
            partial_embedding = self.embeddings[i](torch.as_tensor(index_list, device = self.embeddings[0].weight.device))
            embeddings_sum = torch.add(embeddings_sum, partial_embedding)
        
        return embeddings_sum
    
    
    def forward(self, input_ids: torch.Tensor, counters_states:List = None):
        new_counters_states = []
        pos_embeddings = []
        
        for i, sequence in enumerate(input_ids):
            if counters_states == None or counters_states == 0:
                embeddings_map, new_counters_state = self.get_sequence_embeddings_map(sequence, None)
            else:
                embeddings_map, new_counters_state = self.get_sequence_embeddings_map(sequence, counters_states[i])
            
            pos_embeddings.append(self.get_embeddings_sum(embeddings_map))
            new_counters_states.append(new_counters_state)
           
        print(pos_embeddings)
        raise ag
        return torch.stack(pos_embeddings)


In [9]:
#model.decoder.model.decoder.embed_positions = DimPositionalEmbedding(2, [cell_tokens, row_tokens], [2048, 200, 200], 1024)

In [10]:
model

VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
           

In [11]:
processor.image_processor.size = image_size[::-1] # should be (width, height)
processor.image_processor.do_align_long_axis = False


In [12]:
with open("msg.json", 'w') as out:
        json.dump({'outputs': []}, out, ensure_ascii=False, indent=4)

def write_msg(msg):
    with open("msg.json", encoding="utf-8") as f:
        json_data = json.load(f)
    
    with open("msg.json", 'w') as out:
        json_data['outputs'].append(msg)
        json.dump(json_data, out, ensure_ascii=False, indent=4)

In [13]:
def cel2token(cell):
    if cell['span_type'][10:] != '0000':
        sequence = "<" + cell['span_type'] + ">"
        if cell['content_holder']:
            if cell['row_header'] and cell['col_header']:
                sequence += "<content_row_and_col_header>"
            elif cell['col_header']:
                sequence += "<content_col_header>"
            elif cell['row_header']:
                sequence += "<content_row_header>"
            else:
                sequence += "<content>"
            sequence += cell['content']
    else:
        sequence = ""
        if cell['content_holder']:
            if cell['row_header'] and cell['col_header']:
                sequence += "<row_and_col_header>"
            elif cell['col_header']:
                sequence += "<col_header>"
            elif cell['row_header']:
                sequence += "<row_header>"
            else:
                sequence += "<cell>"
            sequence += cell['content']
        
    return sequence

def row2token(row):
    sequence = "<row>"
    for cell in row:
        sequence += cel2token(cell)
    
    return sequence


def table2token(table):
    sequence = "<table>"
    for row in table:
        sequence += row2token(row)
    
    return sequence


def json2token(json):
    sequence = ""
    if('tables' in json):
        for table in json['tables']:
            sequence += table2token(table)

    return sequence

In [14]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class DonutTableDataset(Dataset):
    def __init__(
        self,
        annotations,
        image_size,
        max_length,
        shuffle = True,
        split = "train",
        ignore_id = -100,
        prompt_end_token = None,
    ):            
        self.annotations = annotations
        
        
        self.image_size = image_size
        self.max_length = max_length
        self.split = split
        self.ignore_id = ignore_id
        
        
    def __len__(self):
        return len(self.annotations)
    
    
    def __getitem__(self, idx):
        
        file_name = self.annotations[idx]
        
        with open(ANN_PATH + file_name + ".json", encoding="utf-8") as f:
            annotation = json.load(f)
        
        image = Image.open(IMAGE_PATH + file_name + IMG_FORMAT)
        
        
        # inputs
        pixel_values = processor(image.convert("RGB"), random_padding=self.split == "train", return_tensors="pt").pixel_values.squeeze()
        pixel_values = pixel_values.squeeze()
        
        target_sequence = json2token(annotation)+"</s>"
        
        input_ids = processor.tokenizer(
            target_sequence,
            add_special_tokens=False,
            max_length= max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"].squeeze(0)

        labels = input_ids.clone()
        
        labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id
        
        
        encoding = dict(pixel_values=pixel_values,
                        labels=labels)
        
        return encoding

In [15]:
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(["<table_extraction>"])[0] 


In [16]:
train_dataset = DonutTableDataset(json_list,
                             max_length = max_length,
                             image_size = image_size)

In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [18]:
#model = torch.load("model_epoch_checkpoint_.bin")
start_epoch = 0
avg_size = 100

In [19]:
import torch
from tqdm.auto import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model.to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=5e-5)


for epoch in range(start_epoch, 1):
    print("Epoch:", epoch+1)
    mean_loss = 0
    mean_smpl_loss = 0 
    model.train()
    for i, batch in enumerate(tqdm(train_dataloader)):
        
        batch = {k: v.to(device) for k, v in batch.items()}
        
        pixel_values = batch["pixel_values"]
        labels = batch["labels"]
        
        
        outputs = model(pixel_values=pixel_values, labels=labels)
        
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        mean_loss += loss.item()   
        mean_smpl_loss += loss.item() 
        if i % avg_size == 0:
            print(str(i) + " Loss: ", mean_smpl_loss/avg_size)
            write_msg("batch " + str(i) +" loss: "+ str(mean_smpl_loss/avg_size))
            mean_smpl_loss = 0 
        
        if i % 10000 == 0:
             model.save_pretrained("../../pubtabnet/TML-lre-5-checkpoint")
    
    model.save_pretrained("../../pubtabnet/TML-lre-5-checkpoint")
    print("Epoch's mean loss: ", mean_loss/len(train_dataloader))
    
    write_msg("Epoch checkpointed: " + str(epoch+1) +" \n"+
              "Epoch's mean Loss: " + str(mean_loss/len(train_dataloader)))

Epoch: 1


  0%|          | 0/125194 [00:00<?, ?it/s]

../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [1,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [1,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [1,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [1,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [1,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [1,0,0], thread: [69,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [1,0,0], thread: 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
len(train_dataset)

In [ ]:
model.save_pretrained("../../pubtabnet/model-minimal-3D-epoch1-lr4")

In [ ]:
torch.save(model.state_dict(), "../../pubtabnet/model-minimal-3D-epoch1-pytorch")

In [ ]:
processor.save_pretrained("../../pubtabnet/Donut_PubTables_TML_Processor")

In [ ]:
sys.path.insert(0,'../src/backup/dimbart')

from modeling_dimbart import DiMBartForCausalLM
from configuration_dimbart import DiMBartConfig

In [ ]:
DiMBartConfig()

In [ ]:
decoder_config = model.config.decoder

In [ ]:
decoder_config

In [ ]:
dimBart_config = DiMBartConfig(**decoder_config.__dict__)

In [ ]:
dimbart = model.decoder

In [ ]:
dimbart.config = dimBart_config
dimbart.model.config = dimBart_config
dimbart.model.decoder.config = dimBart_config

In [ ]:
dimbart.model.decoder.config

In [ ]:
dimbart.config.pos_counters = [cell_tokens, row_tokens]

In [ ]:
dimbart.config

In [ ]:
dimbart.save_pretrained("tabeleiro/dimbart_decoder")

In [ ]:
model.encoder.save_pretrained("tabeleiro/donut_encoder")